## Building a Scalable Agentic Solution with Object-Oriented Principles

Designing scalable LLM solutions requires more than just raw performance of the models — it also demands code that is easy to maintain, extend, and modify as requirements evolve. By leveraging Object-Oriented Programming (OOP) principles, we can create modular, testable systems that will stand the test of time. This Cookbook walks you through an OOP-driven implementation for OpenAI’s Chat Completions API, using design techniques such as inheritance, interfaces, dependency injection, and the factory pattern.

### Overview of Core Components

The solution is provided in the directory `./object_oriented_agents/` you can examine the code to follow along. For brevity, this Cookbook will focus on the overview of the architecture and a concrete implementation for demonstration purpose.

1.Base Classes Through Inheritance
- BaseAgent, ToolInterface, and LanguageModelInterface are abstract classes meant to be extended for specific use cases.
- OpenAILanguageModel is a concrete class implementing LanguageModelInterface, providing a bridge to the OpenAI Chat Completions API.


2.Key Associations and Dependencies
- BaseAgent composes a ChatMessages instance for managing conversation logs (i.e., storing and retrieving messages).
- BaseAgent optionally holds a ToolManager but always requires a LanguageModelInterface.
- ToolManager maintains a set of ToolInterface implementations, uses ChatMessages for context, and delegates API calls to the language model.
- OpenAILanguageModel relies on OpenAIClientFactory for client creation, ensuring a clean separation of concerns around API-key resolution and client configuration.



The following diagram represents the main classes, interfaces and their relationship.

![UML Diagram](./resources/images/oo_agents_uml_diagram.png)

### OOP Best Practices in Action

1. Abstraction & Interfaces
- Abstract classes (BaseAgent, ToolInterface, LanguageModelInterface) define clear contracts, making it easy to swap in new implementations—such as different language models or tools—without disrupting existing code.
- This flexibility is key to building scalable, future-proof systems.

2. Encapsulation & Single Responsibility
- Each class is designed around a specific concern:
- ChatMessages manages message storage and manipulation.
- ToolManager handles tool registration, definitions, and invocation logic.
- BaseAgent orchestrates user tasks, message flow, and the interactions between the language model and tools.
- OpenAILanguageModel encapsulates direct communication with the OpenAI API.
- Following the Single Responsibility Principle (SRP) ensures that classes remain focused, maintainable, and easy to reason about.

3. Dendency Injection & Inversion
- Dependencies (e.g., LanguageModelInterface, logging utilities, ToolManager) are passed into constructors or provided through interfaces, preventing rigid coupling to concrete implementations.
- This design aligns with the Dependency Inversion Principle, where high-level modules (BaseAgent) depend on abstractions (LanguageModelInterface) rather than concrete classes (OpenAILanguageModel).

4. Factory Pattern
- OpenAIClientFactory centralizes client creation, including any logic for API keys, environment variables, or other configuration details.
- By encapsulating client instantiation, you can easily adjust how clients are created without scattering changes throughout the codebase.

5. Separation of Concerns
- Responsibilities for tool invocation, message handling, and external API communication are each isolated within their own classes.
- This separation yields a system that is straightforward to maintain, debug, and extend over time—an essential trait in scalable architectures.

### Building a Weather Agent with Tool Calling

As an example of a concrete implementation, let's build a Weather agent that can fetch and display weather given a zipcode.

First, let's define the **CheckWeatherTool**. Note that the tool definition for the LLM tool call and the implementation of the tool execution is wrapped in the same class for maintainability.

In [11]:
import sys, os

# Add the parent directory of 'core_classes' to the Python path
sys.path.append(os.path.abspath('resources/object_oriented_agents/'))

In [12]:
import requests
from typing import Dict, Any
from resources.object_oriented_agents.core_classes.tool_interface import ToolInterface

class CheckWeatherTool(ToolInterface):
    """
    A tool to check the latest weather given a zipcode.
    """

    def __init__(self, openweather_api_key: str = None):
        # You can pass an API key for a weather service if you have one.
        # Otherwise, this can be left as None and the run() method can be mocked or adapted.
        self.openweather_api_key = openweather_api_key

    def get_definition(self) -> Dict[str, Any]:
        """
        Return the JSON/dict definition of this tool's function.
        Conforms to OpenAI function calling specifications.
        """
        return {
            "function": {
                "name": "check_weather",
                "description": "Check the current weather for a given US zipcode.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "zipcode": {
                            "type": "string",
                            "description": "The 5-digit US ZIP code for which to check the weather."
                        }
                    },
                    "required": ["zipcode"]
                }
            }
        }

    def run(self, arguments: Dict[str, Any]) -> str:
        """
        Execute the tool using the provided arguments and return a result as a string.
        Here, we'll call the OpenWeather API (if you have a key) or just return a mock result.
        """
        zipcode = arguments.get("zipcode")
        if not zipcode:
            return "Error: No zipcode provided."

        if not self.openweather_api_key:
            raise ValueError("Error: No API key provided for weather service.")
        try:
            # Example call to OpenWeatherMap (if you have an API key and want real data).
            # Adjust the endpoint and parameters as needed for your chosen weather service.
            url = f"https://api.openweathermap.org/data/2.5/weather"
            params = {
                "zip": zipcode,
                "appid": self.openweather_api_key,
                "units": "imperial"
            }
            response = requests.get(url, params=params)
            response.raise_for_status()
            data = response.json()

            # Extracting simple weather info
            city_name = data.get("name", "Unknown location")
            main_weather = data["weather"][0]["description"]
            temp = data["main"]["temp"]

            return f"The current weather in {city_name} ({zipcode}): {main_weather}, {temp}°F."
        except Exception as e:
            return f"Error when calling weather API: {str(e)}"

Now, let's define an agent to use the tool.

In [13]:
# object_oriented_agents/agents/weather_agent.py

from typing import Optional
import logging

from dotenv import load_dotenv
import os


from resources.object_oriented_agents.core_classes.base_agent import BaseAgent
from resources.object_oriented_agents.core_classes.tool_manager import ToolManager
from resources.object_oriented_agents.utils.logger import get_logger
from resources.object_oriented_agents.services.language_model_interface import LanguageModelInterface
from resources.object_oriented_agents.services.openai_language_model import OpenAILanguageModel

# Set the verbosity level: DEBUG for verbose output, INFO for normal output
app_logger = get_logger("MyApp", level=logging.DEBUG)


# Create a LanguageModelInterface instance using the OpenAILanguageModel
language_model_api_interface = OpenAILanguageModel(api_key=os.getenv("OPENAI_API_KEY"), logger=app_logger)

load_dotenv()
openweather_api_key = os.getenv("OPENWEATHER_API_KEY")  

class WeatherAgent(BaseAgent):
    """
    An agent that can check the weather by calling the CheckWeatherTool.
    """

    def __init__(
        self,
        developer_prompt: str = "You are a Weather Agent. You answer questions about the weather.",
        model_name: str = "gpt-4o",
        logger= app_logger,
        language_model_interface: Optional[LanguageModelInterface] = language_model_api_interface,
        openweather_api_key: Optional[str] = openweather_api_key,
    ):
        super().__init__(developer_prompt, model_name, logger, language_model_interface)
        # Create a tool manager specifically for this agent
        self.tool_manager = ToolManager(logger=self.logger, language_model_interface=self.language_model_interface)
        self.openweather_api_key = openweather_api_key
        # Set up the weather tool
        self.setup_tools()

    def setup_tools(self) -> None:
        """
        Register all the tools needed by this agent, e.g., the weather tool.
        """
        weather_tool = CheckWeatherTool(openweather_api_key=self.openweather_api_key)
        self.tool_manager.register_tool(weather_tool)

In [ ]:
# Create the WeatherAgent
agent = WeatherAgent()

# Example user input
user_question = "What is the weather in 90210 right now?"

# Have the agent handle the user's task
result = agent.task(user_question)

print("Agent response:", result)

In [ ]:
user_question = "What is the weather in 94551 right now?"

# Have the agent handle the user's task
result = agent.task(user_question)

print("Agent response:", result)